In [1]:
import json
import tweepy
import time
import threading
import datetime as dt
from datetime import datetime
import numpy as np
import pandas as pd

from IPython.core.display import HTML
directory = '/home/aakant/Development/WorkingDirectories/JupyterNotebook/'
css = open(directory + 'style-table.css').read() + open(directory + 'style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [2]:
from tweepy import OAuthHandler
 
consumer_key = 'smF3mKQ8aKwySgHC7qUvMtGBi'
consumer_secret = '1fk60NTx8cWuV8QMDs1LdW3YkJjUUjxu2cLeGhDHAsl68raLD2'
access_token = '71018872-Ad3En3sV6NI3XLKdeRG3NmJpqefJN8Apt1KTJGI8Q'
access_secret = '5wO0XztpbiX49ozt0MXdW1B8TYvdG1kPgy8jZSO7SvuxY'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [3]:
twitterSourcesURL="/home/aakant/Development/WorkingDirectories/JupyterNotebook/Capstone/twitter-sources.csv"
twitterSources = pd.read_csv(twitterSourcesURL, header=0)
twitterSources['TwitterID'] = twitterSources['TwitterID'].str.replace('@', '')
twitterSources

,Twitter Source Name,TwitterID
0,The Independent,Independent
1,The Economist,TheEconomist
2,Reuters,Reuters
3,Forbes,Forbes
4,Financial Times,FT
5,Bloomberg News,business
6,The Wall Street Journal,WSJ
7,The Real News,TheRealNews
8,The Independent Media Center,indymedia
9,AlterNet,AlterNet


In [4]:
def show_rate_limit_status():
    rate_limit_status = api.rate_limit_status()
    rate_limit_df = pd.DataFrame(rate_limit_status['resources'])
    rate_limit_df = rate_limit_df.stack()
    rate_limit_df = rate_limit_df.reset_index()
    rate_limit_df = rate_limit_df.drop('level_1',1)
    rate_limit_df['remaining'] = rate_limit_df[0].map(lambda x: x['remaining'])
    rate_limit_df['reset'] = rate_limit_df[0].map(lambda x: x['reset'])
    rate_limit_df['limit'] = rate_limit_df[0].map(lambda x: x['limit'])
    rate_limit_df = rate_limit_df.drop(0,1)
    rate_limit_df['reset'] = list(map(dt.datetime.fromtimestamp, rate_limit_df['reset']))
    return rate_limit_df[rate_limit_df['remaining'] < rate_limit_df['limit']]

In [5]:
def limit_handled(cursor, usernumber):
    flag=True
    count = 1
    while flag:
        try:
            yield cursor.next()
        except (tweepy.TweepError, tweepy.RateLimitError):
            print("Rate Limit Hit, sleeping for 15 minutes. User Number: "+str(usernumber))
            time.sleep(15 * 60)
            
def get_tweets(userList=twitterSources['TwitterID'], theCount=10, startDate=None, endDate=None, numPages=None):
    usernumber = 0
    array = []
    for sourceName in userList:
        usernumber += 1
        if numPages:
            for theTweets in limit_handled(tweepy.Cursor(api.user_timeline, screen_name=sourceName).pages(numPages), usernumber):
                for tweet in theTweets:
                    parsed_json = json.dumps(tweet._json)
                    array.append(parsed_json)
        else:
            for theTweets in limit_handled(tweepy.Cursor(api.user_timeline, screen_name=sourceName).pages(), usernumber):
                for tweet in theTweets:
                    parsed_json = json.dumps(tweet._json)
                    array.append(parsed_json)
        
    return array

In [6]:
def arrayToDF(arrayElement):  
    tempDF = pd.read_json(arrayElement)
    tempDF = tempDF.loc['name']
    return tempDF

In [7]:
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

In [12]:
show_rate_limit_status()

,level_0,remaining,reset,limit
4,/application/rate_limit_status,178,2016-08-13 00:54:46,180


In [9]:
class myThread (threading.Thread):
    def __init__(self, threadID, name):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
    def run(self):
        print ("Starting " + self.name)
        theTweets = get_tweets()
        bigTweetDF = pd.DataFrame.from_records(map(arrayToDF, theTweets))
        bigTweetDF.to_json(directory+'AllTweets13August.json')
        print ("Exiting " + self.name)

In [10]:
thread1 = myThread(1,"TwitterFeedRetrieval")
thread1.start()

Starting TwitterFeedRetrieval
Rate Limit Hit, sleeping for 15 minutes. User Number: 1
Rate Limit Hit, sleeping for 15 minutes. User Number: 2
Rate Limit Hit, sleeping for 15 minutes. User Number: 3
Rate Limit Hit, sleeping for 15 minutes. User Number: 3
Rate Limit Hit, sleeping for 15 minutes. User Number: 4
Rate Limit Hit, sleeping for 15 minutes. User Number: 5
Rate Limit Hit, sleeping for 15 minutes. User Number: 5
Rate Limit Hit, sleeping for 15 minutes. User Number: 6
Rate Limit Hit, sleeping for 15 minutes. User Number: 6
Rate Limit Hit, sleeping for 15 minutes. User Number: 7
Rate Limit Hit, sleeping for 15 minutes. User Number: 8
Rate Limit Hit, sleeping for 15 minutes. User Number: 8
Rate Limit Hit, sleeping for 15 minutes. User Number: 9
Rate Limit Hit, sleeping for 15 minutes. User Number: 9
Rate Limit Hit, sleeping for 15 minutes. User Number: 10
Rate Limit Hit, sleeping for 15 minutes. User Number: 12
Rate Limit Hit, sleeping for 15 minutes. User Number: 13
Rate Limit Hit,